In [8]:
import os
import time
import pathlib
import yaml
import math
from datetime import datetime

In [9]:
now = datetime.now()
dt_string = now.strftime("%d%B%Y")
dt_string

'15December2021'

In [10]:
startTime = time.time()
group_runs = 200 # Number of output files aggregated together
ngroups = 35  #This is a number >> max number of aggregations in any given oil type
#------------------------------------------------------------
# Global variables
#------------------------------------------------------------
oil_types = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [11]:
yaml_file=pathlib.Path('/scratch/rmueller/MIDOSS/Results/MOHID_results_locations_27112021_12:05:17.yaml')


with yaml_file.open("rt") as f:
    run_paths = yaml.safe_load(f) 

In [12]:
run_paths['dilbit']

['/scratch/dlatorne/MIDOSS/runs/monte-carlo/near-BP_18th-100_2021-09-30T145416/results/near-BP_18th-100-94/Lagrangian_dilbit-94_near-BP_18th-100-94.nc']

In [17]:
nc_in

'/scratch/dlatorne/MIDOSS/runs/monte-carlo/near-BP_13th-100_2021-09-25T163257/results/near-BP_13th-100-61/Lagrangian_akns-61_near-BP_13th-100-61.nc'

In [9]:
import xarray
data = xarray.open_dataset(nc_in, engine='h5netcdf')

In [23]:
run

'/scratch/dlatorne/MIDOSS/runs/monte-carlo/near-BP_13th-100_2021-09-25T163257/results/near-BP_13th-100-61/Lagrangian_akns-61_near-BP_13th-100-61.nc'

In [50]:
os.path.dirname(os.path.abspath(run))

'/scratch/dlatorne/MIDOSS/runs/monte-carlo/near-BP_13th-100_2021-09-25T163257/results/near-BP_13th-100-61'

In [44]:
path = '/scratch/dlatorne/MIDOSS/runs/monte-carlo/near-BP_13th-100_2021-09-25T163257/results/near-BP_13th-100-61/'
run = '/scratch/dlatorne/MIDOSS/runs/monte-carlo/near-BP_13th-100_2021-09-25T163257/results/near-BP_13th-100-61/Lagrangian_akns-61_near-BP_13th-100-61.nc'
nrun = run.split('/')[-1].split('-')[-1][:-3]
oil_tag = run.split('/')[-1].split('_')[1].split('-')[0]
dat_file_path=os.path.join(path,f'Model-{nrun}.dat')
Lagrangian_file = dir+f'Lagrangian_{oil_tag}-{nrun}.dat'
Lagrangian_file

'/scratch/dlatorne/MIDOSS/runs/monte-carlo/near-BP_13th-100_2021-09-25T163257/results/near-BP_13th-100-61/Lagrangian_akns-61.dat'

In [45]:
with open(Lagrangian_file, 'r') as f:
    for line in f:
        if 'POINT_VOLUME              :' in line: 
            # select spill volume quantity after ':'
            spillvolume = line.split(':')[-1].split('\n')[0]
        if 'POSITION_COORDINATES      :' in line:
            spill_location = line.split(':')[-1].split('\n')[0]

In [58]:
(float(spill_location.split(' ')[1]),float(spill_location.split(' ')[2]))

(-122.58232625325522, 48.51557540893555)

In [55]:
float(spillvolume)

14119.655008200005

In [60]:
import numpy
latitude=numpy.zeros(2)
longitude=numpy.zeros(2)
latitude[1] = 48.51557540893555

In [62]:
latitude[1]

48.51557540893555

In [ ]:
with open(Lagrangian_file, 'r') as f:
    for line in f:
        if 'POINT_VOLUME              :' in line: 
            # select spill volume quantity after ':'
            spillvolume = line.split(':')[-1].split('\n')[0]
            # convert to float for "sort_values" to work properly
            volume[oil]['volume'].append(float(spillvolume))

In [31]:
import math
yaml_file=pathlib.Path('/scratch/rmueller/MIDOSS/Results/MOHID_results_locations_27112021_12:05:17.yaml')
#------------------------------------------------------------
# Output directory
#------------------------------------------------------------
# add date tag to sbatch .list filename
now = datetime.now()
dt_string = now.strftime("%d%B%Y")
#output_list_dir=f'/scratch/rmueller/MIDOSS/Results/{dt_string}' 
output_list_dir='/scratch/rmueller/MIDOSS/Results/ByOilType/13December2021'
# create output directory if it doesn't already exist
try:
    os.makedir(output_list_dir)
except:
    print(f'{output_list_dir} already exists')
sbatch_file = pathlib.Path(
    output_list_dir,
    f'aggregate_{group_runs}_oil_spills_{dt_string}.list')
#------------------------------------------------------------
# Load yaml file name with list of output netcdf files to aggregate
#------------------------------------------------------------
with yaml_file.open("rt") as f:
    run_paths = yaml.safe_load(f)   
n_iter={}
for oil in oil_types:
    n_iter[oil]=max(1,math.ceil(len(run_paths[oil])/group_runs))
#------------------------------------------------------------
# Iterate through oil types and aggregate SOILED model output
# in batches of "group_runs" number of files
#------------------------------------------------------------
all_iter = 0
with open(sbatch_file, 'w') as f:
    for oil in oil_types: # loop through oils
        this_iter = 1
        print(f'*** {oil} (Number of iterations: {n_iter[oil]}) ***')
        # loop through specified number of groups
        while (this_iter <= ngroups) & (n_iter[oil]>=this_iter): 
            print(f'Iteration: {this_iter} ')
            first = (this_iter-1) * group_runs 
            last = this_iter*group_runs
            #------------------------------------------------------------
            # write to file the sbatch submission command
            f.write(f"python aggregate_oil_spills.py '{yaml_file}' {oil} {first} {last} '{output_list_dir}'\n")
            this_iter += 1
            all_iter += 1
        executionTime = (time.time() - startTime)
        print(f'Execution time in minutes for {oil}: {executionTime/60:.2f}')
f.close

/scratch/rmueller/MIDOSS/Results/ByOilType/13December2021 already exists
*** akns (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for akns: 56.48
*** bunker (Number of iterations: 17) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Iteration: 6 
Iteration: 7 
Iteration: 8 
Iteration: 9 
Iteration: 10 
Iteration: 11 
Iteration: 12 
Iteration: 13 
Iteration: 14 
Iteration: 15 
Iteration: 16 
Iteration: 17 
Execution time in minutes for bunker: 56.48
*** dilbit (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for dilbit: 56.48
*** jet (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for jet: 56.48
*** diesel (Number of iterations: 32) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Iteration: 6 
Iteration: 7 
Iteration: 8 
Iteration: 9 
Iteration: 10 
Iteration: 11 
Iteration: 12 
Iteration: 13 
Iteration: 14 
Iteration: 15 
Iteration: 16 
Iteration: 17 
Iteration: 18 
Itera

<function TextIOWrapper.close()>

In [32]:
total_files = 0
for oil in oil_types:
    total_files += len(run_paths[oil])
total_files

9823

In [28]:
yaml_file=pathlib.Path('/scratch/rmueller/MIDOSS/Results/MOHID_results_locationsByMonth_14122021_01:09:57.yaml')
#------------------------------------------------------------
# Output directory
#------------------------------------------------------------
# add date tag to sbatch .list filename
now = datetime.now()
dt_string = now.strftime("%d%B%Y")
output_list_dir=f'/scratch/rmueller/MIDOSS/Results/ByMonth/{dt_string}' 

# create output directory if it doesn't already exist
try:
    os.makedirs(output_list_dir)
except:
    print(f'{output_list_dir} already exists')
sbatch_file = pathlib.Path(
    output_list_dir,
    f'aggregate_{group_runs}_oil_spills_{dt_string}.list')
#------------------------------------------------------------
# Load yaml file name with list of output netcdf files to aggregate
#------------------------------------------------------------
with yaml_file.open("rt") as f:
    run_paths = yaml.safe_load(f)   
n_iter={}
for month in months:
    n_iter[month]=max(1,math.ceil(len(run_paths[month])/group_runs))
#------------------------------------------------------------
# Iterate through oil types and aggregate SOILED model output
# in batches of "group_runs" number of files
#------------------------------------------------------------
all_iter = 0
with open(sbatch_file, 'w') as f:
    for month in months: # loop through oils
        this_iter = 1
        print(f'*** {month} (Number of iterations: {n_iter[month]}) ***')
        # loop through specified number of groups
        while (this_iter <= ngroups) & (n_iter[month]>=this_iter): 
            print(f'Iteration: {this_iter} ')
            first = (this_iter-1) * group_runs 
            last = this_iter*group_runs
            #------------------------------------------------------------
            # write to file the sbatch submission command
            f.write(f"python aggregate_oil_spills.py '{yaml_file}' {month} {first} {last} '{output_list_dir}'\n")
            this_iter += 1
            all_iter += 1
        executionTime = (time.time() - startTime)
        print(f'Execution time in minutes for {month}: {executionTime/60:.2f}')
f.close

/scratch/rmueller/MIDOSS/Results/ByMonth/14December2021 already exists
*** Jan (Number of iterations: 4) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Execution time in minutes for Jan: 44.62
*** Feb (Number of iterations: 4) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Execution time in minutes for Feb: 44.62
*** Mar (Number of iterations: 4) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Execution time in minutes for Mar: 44.62
*** Apr (Number of iterations: 4) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Execution time in minutes for Apr: 44.62
*** May (Number of iterations: 5) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Execution time in minutes for May: 44.62
*** Jun (Number of iterations: 5) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Execution time in minutes for Jun: 44.62
*** Jul (Number of iterations: 5) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 

<function TextIOWrapper.close()>

In [30]:
total_files = 0
for month in months:
    total_files += len(run_paths[month])
total_files

9823

### Create a secondary file for runs that didn't complete 

##### By month

In [3]:
yaml_file=pathlib.Path('/scratch/rmueller/MIDOSS/Results/MOHID_results_locationsByMonth_14122021_01:09:57.yaml')
output_list_dir=f'/scratch/rmueller/MIDOSS/Results/ByMonth/14December2021' 
sbatch_file = pathlib.Path(
    output_list_dir,
    f'aggregate_200_oil_spills_14December2021_LONG.list')
with yaml_file.open("rt") as f:
    run_paths = yaml.safe_load(f)   
n_iter={}
for month in months:
    n_iter[month]=max(1,math.ceil(len(run_paths[month])/group_runs))

In [5]:
#------------------------------------------------------------
# Iterate through oil types and aggregate SOILED model output
# in batches of "group_runs" number of files
#------------------------------------------------------------
with open(sbatch_file, 'w') as f:
    for month in months: # loop through oils
        this_iter = 1
        print(f'*** {month} (Number of iterations: {n_iter[month]}) ***')
        # loop through specified number of groups
        while (this_iter <= ngroups) & (n_iter[month]>=this_iter): 
            print(f'Iteration: {this_iter} ')
            first = (this_iter-1) * group_runs 
            last = this_iter*group_runs
            netcdf_file = pathlib.Path(
                    output_list_dir,
                    f'beaching_{month}_{first}-{last}.nc')
            if os.path.exists(netcdf_file)==False:
                f.write(f"python aggregate_oil_spills.py '{yaml_file}' {month} {first} {last} '{output_list_dir}'\n")
            this_iter += 1
        executionTime = (time.time() - startTime)
        print(f'Execution time in minutes for {month}: {executionTime/60:.2f}')
f.close

*** Jan (Number of iterations: 4) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Execution time in minutes for Jan: 0.74
*** Feb (Number of iterations: 4) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Execution time in minutes for Feb: 0.74
*** Mar (Number of iterations: 4) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Execution time in minutes for Mar: 0.75
*** Apr (Number of iterations: 4) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Execution time in minutes for Apr: 0.75
*** May (Number of iterations: 5) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Execution time in minutes for May: 0.75
*** Jun (Number of iterations: 5) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Execution time in minutes for Jun: 0.75
*** Jul (Number of iterations: 5) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Execution time in minutes for Jul: 0.75
*** Aug (Number of i

<function TextIOWrapper.close()>

##### By oil type

In [6]:
output_list_dir=f'/scratch/rmueller/MIDOSS/Results/ByOilType/13December2021' 
yaml_file=pathlib.Path('/scratch/rmueller/MIDOSS/Results/MOHID_results_locations_27112021_12:05:17.yaml')
sbatch_file = pathlib.Path(
    output_list_dir,
    f'aggregate_oil_spills_LONG2_14December2021.list')
with yaml_file.open("rt") as f:
    run_paths = yaml.safe_load(f)   
n_iter={}
for oil in oil_types:
    n_iter[oil]=max(1,len(run_paths[oil])/group_runs)


In [7]:
#------------------------------------------------------------
# Iterate through oil types and aggregate SOILED model output
# in batches of "group_runs" number of files
#------------------------------------------------------------
with open(sbatch_file, 'w') as f:
    for oil in oil_types: # loop through oils
        this_iter = 1
        print(f'*** {oil} (Number of iterations: {n_iter[oil]}) ***')
        # loop through specified number of groups
        while (this_iter <= ngroups) & (n_iter[oil]>=this_iter): 
            print(f'Iteration: {this_iter} ')
            first = (this_iter-1) * group_runs 
            last = this_iter*group_runs
            netcdf_file = pathlib.Path(
                    output_list_dir,
                    f'beaching_{oil}_{first}-{last}.nc')
            if os.path.exists(netcdf_file)==False:
                f.write(f"python aggregate_oil_spills.py '{yaml_file}' {oil} {first} {last} '{output_list_dir}'\n")
            this_iter += 1
        executionTime = (time.time() - startTime)
        print(f'Execution time in minutes for {oil}: {executionTime/60:.2f}')
f.close

*** akns (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for akns: 0.40
*** bunker (Number of iterations: 16.74) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Iteration: 6 
Iteration: 7 
Iteration: 8 
Iteration: 9 
Iteration: 10 
Iteration: 11 
Iteration: 12 
Iteration: 13 
Iteration: 14 
Iteration: 15 
Iteration: 16 
Execution time in minutes for bunker: 0.40
*** dilbit (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for dilbit: 0.40
*** jet (Number of iterations: 1) ***
Iteration: 1 
Execution time in minutes for jet: 0.40
*** diesel (Number of iterations: 31.195) ***
Iteration: 1 
Iteration: 2 
Iteration: 3 
Iteration: 4 
Iteration: 5 
Iteration: 6 
Iteration: 7 
Iteration: 8 
Iteration: 9 
Iteration: 10 
Iteration: 11 
Iteration: 12 
Iteration: 13 
Iteration: 14 
Iteration: 15 
Iteration: 16 
Iteration: 17 
Iteration: 18 
Iteration: 19 
Iteration: 20 
Iteration: 21 
Iteration: 22 
Iteration: 23 
Iteration: 24 


<function TextIOWrapper.close()>